In [1]:
batch_size = 32
lr = 10**(-4)
n_epochs = 30
fraction_splits = [0.6,0.4 ]

In [2]:
# fill the cache and print informations about caching in other notebook
from pathlib import Path
from ploomber_engine.ipython import PloomberClient

path_training_setup = "./training_setup.py"

import utils_notebook
import psutil
from tqdm import tqdm
with utils_notebook.NotebookPyExecutor(path_training_setup,remove_tagged_cells="parameters",
                                      truncate = None,location = '/root/.cache/keypoints',
                                       clear_cache = False,batch_size=batch_size,
                                       fraction_splits = fraction_splits,lr=lr) as nb_exec:
    trainer = nb_exec.namespace["trainer"]
    validator = nb_exec.namespace["validator"]

    train_video_dataloader = nb_exec.namespace["train_video_dataloader"]
    val_video_dataloader = nb_exec.namespace["val_video_dataloader"]
    print(nb_exec.path_training_setup_as_notebook)
    assert Path(nb_exec.path_training_setup_as_notebook).exists()

free gpu memory     : 5.8G
allocated gpu memory     : 164.0M
free ram 17.7G
used ram 3.9G


Executing cell: 8:  38%|█████████▍               | 6/16 [00:02<00:03,  2.98it/s]<ipython-input-1-9e3b603a6402>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  kpts = torch.tensor(kpts)
<ipython-input-1-9e3b603a6402>:18: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  is_detection_present = torch.tensor(is_detection_present)
Executing cell: 16: 100%|███████████████████████| 16/16 [00:04<00:00,  3.33it/s]

/tmp/tmp28q7b3q_/training_setup.ipynb
free gpu memory     : 5.4G
allocated gpu memory     : 632.2M
free ram 16.7G
used ram 4.8G


In [8]:
import numpy as np
nb_iter_per_lr = 4
total_nb_iter = 500

nb_lr_change = int(total_nb_iter/nb_iter_per_lr)
max_val = 10**(-2.0)
min_val = 10**(-9)
gamma = np.exp(np.log(max_val/min_val)/(total_nb_iter/nb_iter_per_lr))

state_dict = trainer.optimizer.state_dict()
state_dict["param_groups"][0]["lr"] = min_val
trainer.optimizer.load_state_dict(state_dict)

gamma,trainer.optimizer.state_dict()["param_groups"][0]["lr"]

(1.1376272858234309, 1e-09)

In [11]:
from torch.utils.tensorboard import SummaryWriter

import datetime
now = datetime.datetime.now()
experiment_time = now.strftime('%Y-%m-%dT%H:%M:%S') + ('-%02d' % (now.microsecond / 10000))
writer = SummaryWriter(f"runs/lr_range_test")#+{experiment_time}")
print(writer.log_dir)

runs/lr_range_test


In [12]:
from torch.optim import lr_scheduler

scheduler = lr_scheduler.ExponentialLR(trainer.optimizer,gamma=gamma)
for train_batch,val_batch in zip(train_video_dataloader,val_video_dataloader):
    train_metrics_vals = trainer.iterate_on_batch(train_batch)
    val_metrics_vals = validator.iterate_on_batch(val_batch)
    writer.add_scalars("LRRangetest",
                           {"loss":float(train_metrics_vals["loss"]),
                           "lr":scheduler.get_last_lr()[0]}
                           ,nb_iter_per_lr*i+j)
        

value for metric accuracy is :  0.1875
value for metric loss is :  0.05946123227477074


KeyboardInterrupt: 